In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LogisticRegression
import pickle

In [30]:
df = pd.read_excel(r'D:\DS\DS-CSFB\MORED_AUDIT_SURAT_practise.xlsx')
df.head()

,MRBTS,LNBTS,MOPR,MORED,id,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg,redirGeranArfcnStructL,Item-redirGeranArfcnStructL-redirGeranArfcnPrio,Item-redirGeranArfcnStructL-redirGeranArfcnValue,redirGeranBandIndicator,redirFreqUtraTDDL,Remarks
0,791597,791597,1,1,NaN,2.0,1,1,1309,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
1,791622,791622,1,1,NaN,2.0,1,1,1309,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
2,791643,791643,1,1,NaN,2.0,1,1,1309,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
3,791690,791690,1,1,NaN,2.0,1,2,41140,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
4,791694,791694,1,1,NaN,2.0,1,2,41140,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5128 entries, 0 to 5127
Data columns (total 19 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0          MRBTS                                             5128 non-null   int64  
 1          LNBTS                                             5128 non-null   int64  
 2          MOPR                                              5128 non-null   int64  
 3          MORED                                             5128 non-null   int64  
 4   id                                                       0 non-null      float64
 5          addUtranSIToRedirMsg                              2644 non-null   float64
 6          csFallBPrio                                       5128 non-null   int64  
 7          emerCallPrio                                      5128 non-null   int64  
 8          redirFreqEutra       

In [32]:
df.columns = df.columns.str.replace(' ','')
df.head()

,MRBTS,LNBTS,MOPR,MORED,id,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg,redirGeranArfcnStructL,Item-redirGeranArfcnStructL-redirGeranArfcnPrio,Item-redirGeranArfcnStructL-redirGeranArfcnValue,redirGeranBandIndicator,redirFreqUtraTDDL,Remarks
0,791597,791597,1,1,NaN,2.0,1,1,1309,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
1,791622,791622,1,1,NaN,2.0,1,1,1309,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
2,791643,791643,1,1,NaN,2.0,1,1,1309,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
3,791690,791690,1,1,NaN,2.0,1,2,41140,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error
4,791694,791694,1,1,NaN,2.0,1,2,41140,NaN,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,NaN,Error


In [33]:
df.shape

(5128, 19)

In [34]:
class Preprocessor:

    def remove_columns(self,data,columns):
        self.data=data
        self.columns=columns
        try:
            self.useful_data=self.data.drop(self.columns, axis=1) # drop the labels specified in the columns
            print('Column removal Successful.Exited the remove_columns method of the Preprocessor class')
            return self.useful_data
        except Exception as e:
            return print('Error %s' % e)
            raise Exception()

    def handle_missing_values(self, data):
        self.data= data
        try:
            self.data.fillna(0, inplace=True)
            print('Missing values replace with 0 in all columns')
            return self.data
        except Exception as e:
            print('Exception message:  ' + str(e))
            raise Exception()
    
    def handle_categorical(self, data):
        self.data = data
        try:
            cleanup = {'OK' : 1, 'Error' : 0}
            self.data.replace(cleanup, inplace= True)
            return self.data
            print('categorical data is handling done')
        except Exception as e:
            print('Exception message % s' % e)
            raise Exception()

In [35]:
columns= ['id', 'MRBTS','redirGeranArfcnStructL','Item-redirGeranArfcnStructL-redirGeranArfcnPrio', 'Item-redirGeranArfcnStructL-redirGeranArfcnValue','redirFreqUtraTDDL','MOPR','redirGeranBandIndicator']

In [36]:
a1= Preprocessor()

In [37]:
df2 = a1.handle_missing_values(df)

Missing values replace with 0 in all columns


In [38]:
df2.isnull().sum()

MRBTS                                               0
LNBTS                                               0
MOPR                                                0
MORED                                               0
id                                                  0
addUtranSIToRedirMsg                                0
csFallBPrio                                         0
emerCallPrio                                        0
redirFreqEutra                                      0
redirFreqUtra                                       0
redirRAT                                            0
redirectPrio                                        0
addGsmSIToRedirMsg                                  0
redirGeranArfcnStructL                              0
Item-redirGeranArfcnStructL-redirGeranArfcnPrio     0
Item-redirGeranArfcnStructL-redirGeranArfcnValue    0
redirGeranBandIndicator                             0
redirFreqUtraTDDL                                   0
Remarks                     

In [39]:
df3 = a1.handle_categorical(df2)

In [40]:
df3.head()

,MRBTS,LNBTS,MOPR,MORED,id,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg,redirGeranArfcnStructL,Item-redirGeranArfcnStructL-redirGeranArfcnPrio,Item-redirGeranArfcnStructL-redirGeranArfcnValue,redirGeranBandIndicator,redirFreqUtraTDDL,Remarks
0,791597,791597,1,1,0.0,2.0,1,1,1309,0.0,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,0.0,0
1,791622,791622,1,1,0.0,2.0,1,1,1309,0.0,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,0.0,0
2,791643,791643,1,1,0.0,2.0,1,1,1309,0.0,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,0.0,0
3,791690,791690,1,1,0.0,2.0,1,2,41140,0.0,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,0.0,0
4,791694,791694,1,1,0.0,2.0,1,2,41140,0.0,1,1,2.0,List,1;10;2;3;4;5;6;7;8;9,26;35;27;28;29;30;31;32;33;34,0.0,0.0,0


In [41]:
print(type(df))
print(type(df2))
print(type(df3))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [42]:
print(df.shape)
print(df2.shape)
print(df3.shape)

(5128, 19)
(5128, 19)
(5128, 19)


In [43]:
df4= a1.remove_columns(df3,columns)

Column removal Successful.Exited the remove_columns method of the Preprocessor class


In [44]:
df4.head()

,LNBTS,MORED,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg,Remarks
0,791597,1,2.0,1,1,1309,0.0,1,1,2.0,0
1,791622,1,2.0,1,1,1309,0.0,1,1,2.0,0
2,791643,1,2.0,1,1,1309,0.0,1,1,2.0,0
3,791690,1,2.0,1,2,41140,0.0,1,1,2.0,0
4,791694,1,2.0,1,2,41140,0.0,1,1,2.0,0


In [45]:
df4.shape

(5128, 11)

In [46]:
x = df4.drop(columns = 'Remarks')
x.head()

,LNBTS,MORED,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg
0,791597,1,2.0,1,1,1309,0.0,1,1,2.0
1,791622,1,2.0,1,1,1309,0.0,1,1,2.0
2,791643,1,2.0,1,1,1309,0.0,1,1,2.0
3,791690,1,2.0,1,2,41140,0.0,1,1,2.0
4,791694,1,2.0,1,2,41140,0.0,1,1,2.0


In [47]:
x.shape

(5128, 10)

In [48]:
y = df4['Remarks']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Remarks, dtype: int64

In [49]:
#x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.40)
log_reg = LogisticRegression()
log_reg.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [50]:
y_pred = log_reg.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.8806042884990254

In [27]:
Des_tree1 = DecisionTreeClassifier(min_samples_leaf=3, random_state=100)
Des_tree1.fit(x, y)
#Des_tree1.score(x_test,y_test)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=100, splitter='best')

In [58]:
import pickle
with open( 'DTmodel_Pred_MOREDV6.sav', 'wb') as f:
    pickle.dump(Des_tree1,f)

In [56]:
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)
clf.fit(x_train, y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)

In [57]:
# performing predictions on the test dataset 
y_pred = clf.predict(x_test) 
  
# metrics are used to find accuracy or error 
from sklearn import metrics   
print() 
  
# using metrics module for accuracy calculation 
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.9970760233918129
